In [1]:
import pandas as pd


from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense


In [4]:
df = pd.read_csv('../data_cleaning//cleaned_ba_data.csv')

In [55]:
y = df.verified
y = y.astype(int)

In [5]:
corpus = df['corpus']

In [21]:
# count the longest list in the dataset

def longest_list(lst_of_lsts):
    max_len = 0
    longest_lst = None
    for lst in lst_of_lsts:
        if len(lst) > max_len:
            max_len = len(lst)
            # longest_lst = lst
    return max_len

In [22]:
longest_list(corpus)

2395

In [44]:
# vocabulary size
voc_size = 50000

# sentence length
sent_len = 2400

#  one hot encoding
ohr = [one_hot(words, voc_size) for words in corpus]


In [47]:
#  embedding representation: 
embedded_doc = pad_sequences(ohr, padding='pre', maxlen=sent_len)
print(embedded_doc)

[[    0     0     0 ... 27818 25793 12996]
 [    0     0     0 ... 41400 19156 16467]
 [    0     0     0 ... 43459 43459 48410]
 ...
 [    0     0     0 ... 48050 42287 25739]
 [    0     0     0 ... 10314  3303 27116]
 [    0     0     0 ... 27188 26346 45285]]


In [48]:
len(embedded_doc[2])

2400

In [49]:
#  creating the model

embedded_vec_features = 40 #feature representation
model = Sequential()
model.add(Embedding(voc_size, embedded_vec_features,input_length=sent_len))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 2400, 40)          2000000   
                                                                 
 lstm_1 (LSTM)               (None, 100)               56400     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 2056501 (7.84 MB)
Trainable params: 2056501 (7.84 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [59]:
len(embedded_doc), len(y)

(3593, 3593)

In [63]:
X = df[[ 'country',  'stars', 'corpus']]
X.isnull().sum()

In [74]:
import numpy as np
X_final = np.array(embedded_doc)
y_final = np.array(y)

In [75]:
X_final.shape, y_final.shape

((3593, 2400), (3593,))

In [82]:
from sklearn.model_selection import train_test_split

In [83]:
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33)

In [ ]:
# fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5)